## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.7035,30.7861,57.09,83,12,10.67,few clouds
1,1,Portland,US,45.5234,-122.6762,65.53,71,100,4.00,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,72.95,67,82,9.13,broken clouds
3,3,Sabak,MY,3.7675,100.9875,81.55,83,66,4.83,broken clouds
4,4,Walvis Bay,NaN,-22.9575,14.5053,52.75,46,0,10.36,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pref_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & (city_data_df["Max Temp"]>=min_temp)]
pref_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.7035,30.7861,57.09,83,12,10.67,few clouds
1,1,Portland,US,45.5234,-122.6762,65.53,71,100,4.00,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,72.95,67,82,9.13,broken clouds
3,3,Sabak,MY,3.7675,100.9875,81.55,83,66,4.83,broken clouds
4,4,Walvis Bay,NaN,-22.9575,14.5053,52.75,46,0,10.36,clear sky
5,5,Zhigansk,RU,66.7697,123.3711,57.81,53,7,18.05,clear sky
6,6,Vaini,TO,-21.2000,-175.2000,78.96,78,75,5.75,broken clouds
10,10,Alta Floresta,BR,-9.8756,-56.0861,73.26,72,44,2.53,scattered clouds
13,13,Novyy Buyan,RU,53.6861,50.0469,71.76,69,91,4.88,overcast clouds
16,16,Atuona,PF,-9.8000,-139.0333,77.88,69,68,19.24,broken clouds


In [10]:
# 4a. Determine if there are any empty rows.
pref_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pref_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nikolskoye,RU,57.09,few clouds,59.7035,30.7861,
1,Portland,US,65.53,overcast clouds,45.5234,-122.6762,
2,Rikitea,PF,72.95,broken clouds,-23.1203,-134.9692,
3,Sabak,MY,81.55,broken clouds,3.7675,100.9875,
5,Zhigansk,RU,57.81,clear sky,66.7697,123.3711,
6,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
10,Alta Floresta,BR,73.26,scattered clouds,-9.8756,-56.0861,
13,Novyy Buyan,RU,71.76,overcast clouds,53.6861,50.0469,
16,Atuona,PF,77.88,broken clouds,-9.8000,-139.0333,
17,Gisborne,NZ,59.72,broken clouds,-38.6533,178.0042,


In [42]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
        pass
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [65]:
# 7. View the DataFrame and verify that the hotel names have been added
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nikolskoye,RU,57.09,few clouds,59.7035,30.7861,Tourist House - Sablino
1,Portland,US,65.53,overcast clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
2,Rikitea,PF,72.95,broken clouds,-23.1203,-134.9692,People ThankYou
3,Sabak,MY,81.55,broken clouds,3.7675,100.9875,Homestay Nur Kasih Inap Desa
5,Zhigansk,RU,57.81,clear sky,66.7697,123.3711,Gostinichnyy Kompleks Aylgy
6,Vaini,TO,78.96,broken clouds,-21.2,-175.2,Keleti Beach Resort
10,Alta Floresta,BR,73.26,scattered clouds,-9.8756,-56.0861,Floresta Amazonica Hotel
13,Novyy Buyan,RU,71.76,overcast clouds,53.6861,50.0469,“Домик Пчеловода”. Загородное эко-шале
16,Atuona,PF,77.88,broken clouds,-9.8,-139.0333,Villa Enata
17,Gisborne,NZ,59.72,broken clouds,-38.6533,178.0042,ASURE Alfresco Motor Lodge


In [67]:
#7 check for null values
hotel_df["Hotel Name"].isnull().values.sum()

0

In [71]:
# Import dependencies
import numpy as np

In [72]:
# Replace "empty" rows with NaN
hotel_df["Hotel Name"].replace("", np.nan, inplace = True)
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nikolskoye,RU,57.09,few clouds,59.7035,30.7861,Tourist House - Sablino
1,Portland,US,65.53,overcast clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
2,Rikitea,PF,72.95,broken clouds,-23.1203,-134.9692,People ThankYou
3,Sabak,MY,81.55,broken clouds,3.7675,100.9875,Homestay Nur Kasih Inap Desa
5,Zhigansk,RU,57.81,clear sky,66.7697,123.3711,Gostinichnyy Kompleks Aylgy
6,Vaini,TO,78.96,broken clouds,-21.2,-175.2,Keleti Beach Resort
10,Alta Floresta,BR,73.26,scattered clouds,-9.8756,-56.0861,Floresta Amazonica Hotel
13,Novyy Buyan,RU,71.76,overcast clouds,53.6861,50.0469,“Домик Пчеловода”. Загородное эко-шале
16,Atuona,PF,77.88,broken clouds,-9.8,-139.0333,Villa Enata
17,Gisborne,NZ,59.72,broken clouds,-38.6533,178.0042,ASURE Alfresco Motor Lodge


In [73]:
# drop rows with null values
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nikolskoye,RU,57.09,few clouds,59.7035,30.7861,Tourist House - Sablino
1,Portland,US,65.53,overcast clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
2,Rikitea,PF,72.95,broken clouds,-23.1203,-134.9692,People ThankYou
3,Sabak,MY,81.55,broken clouds,3.7675,100.9875,Homestay Nur Kasih Inap Desa
5,Zhigansk,RU,57.81,clear sky,66.7697,123.3711,Gostinichnyy Kompleks Aylgy
6,Vaini,TO,78.96,broken clouds,-21.2,-175.2,Keleti Beach Resort
10,Alta Floresta,BR,73.26,scattered clouds,-9.8756,-56.0861,Floresta Amazonica Hotel
13,Novyy Buyan,RU,71.76,overcast clouds,53.6861,50.0469,“Домик Пчеловода”. Загородное эко-шале
16,Atuona,PF,77.88,broken clouds,-9.8,-139.0333,Villa Enata
17,Gisborne,NZ,59.72,broken clouds,-38.6533,178.0042,ASURE Alfresco Motor Lodge


In [84]:
# remove record that was disrupting creation of labels / heat map and check that it was removed
clean_hotel_df["Hotel Name"].replace('Отель "БАЛИ"', np.nan, inplace = True)
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.tail(10)

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
671,Namibe,AO,63.81,clear sky,-15.1961,12.1522,Hotel Chik Chik Namibe
672,Bandipur,NP,83.25,clear sky,27.9333,84.4167,Bandipur Guest House
673,Bizhbulyak,RU,68.04,broken clouds,53.6967,54.2642,"Бижбуляк, гостиница ООО ""Домоустроитель"""
675,Kirensk,RU,76.14,overcast clouds,57.7853,108.1119,Kirenskiy Plos
678,Kloulklubed,PW,82.9,moderate rain,7.0419,134.2556,Storyboard Beach Resort
679,Nelson Bay,AU,70.09,clear sky,-32.7167,152.15,Mantra Nelson Bay
680,Mahanoro,MG,71.37,broken clouds,-19.9,48.8,BUNGALOW MAHAMANINA
681,Ornskoldsvik,SE,54.5,overcast clouds,63.2909,18.7153,ファーストホテル スタット
682,Labytnangi,RU,64.9,clear sky,66.6572,66.4183,Priuralye Hotel
683,Kabinda,CD,68.04,scattered clouds,-6.1333,24.4833,Village Mianda Mutole


In [85]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [86]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [88]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))